In [7]:
import torch
import gc
import numpy as np
from torch.utils.data import DataLoader
import time
import sys
sys.path.append("/home/sharfikeg/my_files/retIQA/ret/TReSM")
import torch
from torch import nn
from torch.linalg import vector_norm, lstsq
from torchvision import models
import torchvision.transforms as T
import argparse
from models import Net
import pandas as pd
from PIL import Image
import statistics  
import random
from scipy import stats
from sklearn.model_selection import train_test_split
torch.set_num_threads(24)
import data_loader
import folders
import json
import matplotlib.pyplot as plt

In [8]:
# CONSTANTS
img_num = {
        'csiq':     list(range(0, 30)),
        'kadid10k': list(range(0, 80)),
        'tid2013':  list(range(0, 25)),
        'koniq':    list(range(0, 10073)),
        'spaq':     list(range(0, 11125)),
        'pipal':    list(range(0, 200)),
        'biq':      list(range(0, 11989))
        }

botnet_pretrain="/home/sharfikeg/my_files/VIPNet/pretrained_model/botnet_model_best.pth.tar"
botnet_pretrain_classes=150
my_botnet_pretrain="/home/sharfikeg/my_files/retIQA/dc_ret/my_botnet_pretrain/checkpoint_model_best_heads16.pth"
my_botnet_pretrain_classes=125
logging_path="/home/sharfikeg/my_files/retIQA/dc_ret/DistorsionFeatureExtractor/results.csv"
tres_save_path="/extra_disk_1/sharfikeg/Save_TReS/"

tid2013_botnet_finetune="/home/sharfikeg/my_files/retIQA/dc_ret/finetune_botnet50_tid2013_checkpoints/checkpoint_model_best_heads16.pth"
tid2013_data_path="/home/s-kastryulin/data/tid2013/distorted_images/"
tid2013_ref_path="/home/s-kastryulin/data/tid2013/reference_images/"
tid2013_csv_path="/home/sharfikeg/my_files/retIQA/tid2013/tid2013_info.csv"
tid2013_uni_csv_path="/home/sharfikeg/my_files/retIQA/tid2013/tid2013_uni_info.csv"
tid2013_num_classes=120
k_tid=9

kadid10k_botnet_finetune="/home/sharfikeg/my_files/retIQA/dc_ret/finetune_botnet50_kadid10k_checkpoints/checkpoint_model_best_heads16.pth"
kadid10k_data_path="/home/sharfikeg/my_files/retIQA/kadid10k/distorted_images/"
kadid10k_ref_path="/home/sharfikeg/my_files/retIQA/kadid10k/reference_images/"
kadid10k_csv_path="/home/sharfikeg/my_files/retIQA/kadid10k/kadid10k_info.csv"
kadid10k_uni_csv_path="/home/sharfikeg/my_files/retIQA/kadid10k/kadid10k_uni_info.csv"
kadid10k_num_classes=125
k_kadid=9

csiq_botnet_finetune="/home/sharfikeg/my_files/retIQA/dc_ret/finetune_botnet50_csiq_checkpoints/checkpoint_model_best_heads16.pth"
csiq_data_path="/home/sharfikeg/my_files/retIQA/csiq/distorted_images/"
csiq_ref_path="/home/sharfikeg/my_files/retIQA/csiq/src_imgs/"
csiq_csv_path="/home/sharfikeg/my_files/retIQA/csiq/csiq_info.csv"
csiq_uni_csv_path="/home/sharfikeg/my_files/retIQA/csiq/csiq_uni_info.csv"
csiq_num_classes=30
k_csiq=9

koniq10k_data_path="/home/s-kastryulin/data/koniq10k/512x384/"
koniq10k_csv_path="/home/s-kastryulin/data/koniq10k/koniq10k_info.csv"
koniq10k_big_data_path="/home/s-kastryulin/data/koniq10k/1024x768/"
koniq10k_big_csv_path="/home/s-kastryulin/data/koniq10k/koniq10k_big_info.csv"
koniq10k_uni_csv_path="/home/s-kastryulin/data/koniq10k/koniq10k_uni_info.csv"
k_koniq=30

spaq_data_path="/extra_disk_1/sharfikeg/spaq/TestImage/"
spaq_csv_path="/extra_disk_1/sharfikeg/spaq/spaq_info.csv"
spaq_uni_csv_path="/home/sharfikeg/my_files/extra_disk_1/spaq/spaq_uni_info.csv"
k_spaq=50

biq_data_path="/home/sharfikeg/my_files/extra_disk_1/BIQ2021/Images/"
biq_csv_path="/home/sharfikeg/my_files/extra_disk_1/BIQ2021/biq_info.csv"
biq_uni_csv_path="/home/sharfikeg/my_files/extra_disk_1/BIQ2021/biq_uni_info.csv"
k_biq=30

pipal_data_path="/home/sharfikeg/my_files/extra_disk_1/pipal/train/Train_Dist/"
pipal_ref_path="/home/sharfikeg/my_files/extra_disk_1/pipal/train/Train_Ref/"
pipal_csv_path="/home/sharfikeg/my_files/extra_disk_1/pipal/train/pipal_info.csv"
pipal_uni_csv_path="/home/sharfikeg/my_files/extra_disk_1/pipal/train/pipal_uni_info.csv"
k_pipal=20

def str_2_float_list(pseudolist):
    intermediate = pseudolist.strip('][').split(', ')
    return list(map(float, intermediate))
def str_2_str_list(pseudolist):
    intermediate = pseudolist.strip('][').split(', ')
    return list(map(str, intermediate))

In [3]:
seed=10
train, test = train_test_split(img_num['tid2013'], test_size=0.2, random_state=seed)
print(train, test)
path="/home/s-kastryulin/data/tid2013"
csiq_test = folders.TID2013Folder(
    path, seed=seed, index=test, transform=None, patch_num=50, istrain=False, k=15
)
csiq_train = folders.TID2013Folder(
   path, seed=seed, index=train, transform=None, patch_num=50, istrain=True, k=15
)

[13, 27, 12, 1, 19, 14, 18, 6, 11, 23, 24, 28, 22, 10, 26, 29, 8, 25, 16, 17, 0, 15, 4, 9] [20, 7, 5, 2, 3, 21]


In [6]:
csiq_test.samples[4]

('/home/sharfikeg/my_files/retIQA/csiq/dst_imgs_all/rushmore.AWGN.5.png',
 ["'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/log_seaside.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/aerial_city.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/native_american.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/sunset_sparrow.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/sunsetcolor.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/geckos.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/woman.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/swarm.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/elk.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images/AWGN/monument.AWGN.5.png'",
  "'/home/sharfikeg/my_files/retIQA/csiq/distorted_images

In [26]:
path="/home/sharfikeg/my_files/retIQA/csiq"
seed=10
train, test = train_test_split(img_num['csiq'], test_size=0.2, random_state=seed)
train_loader = data_loader.DataLoader(
    'csiq', path, train, 224, 50, seed, 3, batch_size=53, istrain=True
)

In [27]:
for x, y in train_loader.get_data():
    print(sys.getsizeof(x.storage()), sys.getsizeof(y.storage()))
    break

127647792 896


In [28]:
x.element_size()*x.nelement()

127647744